In [18]:
import numpy as np
import matplotlib.pyplot as plt
import os
path = os.getcwd()
os.chdir('..')
from deepnotes import *
os.chdir(path)

# 使用Pytorch验算卷积和池化的梯度
import torch
import torch.nn as nn
import torch.nn.functional as F

In [59]:
# Hyper-parameters
z_dim = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100

D1 = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid()
)

G1 = nn.Sequential(
    nn.Linear(z_dim, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh()
)

In [60]:
D2 = Sequential(
    Linear(image_size, hidden_size),
    LeakyReLU(0.2),
    Linear(hidden_size, hidden_size),
    LeakyReLU(0.2),
    Linear(hidden_size, 1),
    Sigmoid()
)

D2.modules[0].weight -= D2.modules[0].weight
D2.modules[0].weight += D1[0].weight.T.data.numpy()
D2.modules[0].bias -= D2.modules[0].bias
D2.modules[0].bias += D1[0].bias.data.numpy()

D2.modules[2].weight -= D2.modules[2].weight
D2.modules[2].weight += D1[2].weight.T.data.numpy()
D2.modules[2].bias -= D2.modules[2].bias
D2.modules[2].bias += D1[2].bias.data.numpy()

D2.modules[4].weight -= D2.modules[4].weight
D2.modules[4].weight += D1[4].weight.T.data.numpy()
D2.modules[4].bias -= D2.modules[4].bias
D2.modules[4].bias += D1[4].bias.data.numpy()

In [61]:
G2 = Sequential(
    Linear(z_dim, hidden_size),
    ReLU(),
    Linear(hidden_size, hidden_size),
    ReLU(),
    Linear(hidden_size, image_size),
    Tanh()
)

G2.modules[0].weight -= G2.modules[0].weight
G2.modules[0].weight += G1[0].weight.T.data.numpy()
G2.modules[0].bias -= G2.modules[0].bias
G2.modules[0].bias += G1[0].bias.data.numpy()

G2.modules[2].weight -= G2.modules[2].weight
G2.modules[2].weight += G1[2].weight.T.data.numpy()
G2.modules[2].bias -= G2.modules[2].bias
G2.modules[2].bias += G1[2].bias.data.numpy()

G2.modules[4].weight -= G2.modules[4].weight
G2.modules[4].weight += G1[4].weight.T.data.numpy()
G2.modules[4].bias -= G2.modules[4].bias
G2.modules[4].bias += G1[4].bias.data.numpy()

In [62]:
import tensorflow as tf

# load mnist dataset
(x_train_origin,t_train_origin),(x_test_origin,t_test_origin) = tf.keras.datasets.mnist.load_data()

X_train,X_test = x_train_origin/127.5-1,x_test_origin/127.5-1
batch_size = 100

train_loader = DataLoader(X_train,t_train_origin,batch_size)
test_loader = DataLoader(X_test,t_test_origin,batch_size)

In [63]:
loss_func = BCELoss()

images,_ = train_loader.get_batch()
images = images.reshape(images.shape[0],-1)
real_labels = np.ones((batch_size,1))*0.9
fake_labels = np.zeros((batch_size,1))


D2.zero_grad()
p = D2(images)
d_loss,dp = loss_func(p,real_labels)
D2.backward(dp)

z = np.random.randn(batch_size, z_dim)
fake_images = G2(z)
p = D2(fake_images)
d_loss2,dp = loss_func(p,fake_labels)
dx = D2.backward(dp)
G2.backward(dx)


print(torch.FloatTensor(D2.modules[0]._dw))
print(torch.FloatTensor(G2.modules[0]._dw))

tensor([[ 3.8142e-03,  1.8691e-03, -3.9583e-04,  ..., -3.3681e-04,
         -3.4774e-03,  9.9738e-04],
        [ 3.7846e-03,  1.8627e-03, -3.2522e-04,  ..., -4.7253e-04,
         -3.4235e-03,  1.0052e-03],
        [ 3.8580e-03,  1.8610e-03, -5.8603e-04,  ..., -2.3947e-04,
         -3.5073e-03,  1.0130e-03],
        ...,
        [ 3.6917e-03,  1.8559e-03, -2.7191e-04,  ..., -5.0256e-04,
         -3.4215e-03,  1.0035e-03],
        [ 4.3111e-03,  1.8918e-03, -1.1503e-03,  ...,  3.3949e-04,
         -3.6607e-03,  1.0602e-03],
        [ 4.0914e-03,  1.8685e-03, -8.6573e-04,  ...,  5.8522e-05,
         -3.5890e-03,  1.0387e-03]])
tensor([[ 2.4733e-05, -1.7784e-05,  3.8453e-05,  ..., -5.9857e-06,
          1.1233e-04,  1.5950e-04],
        [ 1.2189e-05,  5.1221e-05,  9.4727e-05,  ..., -1.0480e-04,
         -5.1127e-06, -1.3556e-04],
        [-2.9337e-05, -6.1800e-05, -5.6997e-05,  ..., -9.5516e-05,
          1.3723e-05,  6.8221e-05],
        ...,
        [-2.8875e-05,  1.7340e-05,  7.1711e-05

In [64]:
p = D1(torch.FloatTensor(images))
loss = F.binary_cross_entropy(p,torch.FloatTensor(real_labels))

fake_images = G1(torch.FloatTensor(z))
p = D1(torch.FloatTensor(fake_images))
loss += F.binary_cross_entropy(p,torch.FloatTensor(fake_labels))


loss.backward()
print(D1[0].weight.grad.T)
print(G1[0].weight.grad.T)

tensor([[ 3.8142e-03,  1.8691e-03, -3.9583e-04,  ..., -3.3681e-04,
         -3.4774e-03,  9.9738e-04],
        [ 3.7846e-03,  1.8627e-03, -3.2522e-04,  ..., -4.7253e-04,
         -3.4235e-03,  1.0052e-03],
        [ 3.8580e-03,  1.8610e-03, -5.8603e-04,  ..., -2.3947e-04,
         -3.5073e-03,  1.0130e-03],
        ...,
        [ 3.6917e-03,  1.8559e-03, -2.7191e-04,  ..., -5.0256e-04,
         -3.4215e-03,  1.0035e-03],
        [ 4.3111e-03,  1.8918e-03, -1.1503e-03,  ...,  3.3949e-04,
         -3.6607e-03,  1.0602e-03],
        [ 4.0914e-03,  1.8685e-03, -8.6573e-04,  ...,  5.8522e-05,
         -3.5890e-03,  1.0387e-03]])
tensor([[ 2.4733e-05, -1.7784e-05,  3.8453e-05,  ..., -5.9857e-06,
          1.1233e-04,  1.5950e-04],
        [ 1.2189e-05,  5.1221e-05,  9.4727e-05,  ..., -1.0480e-04,
         -5.1126e-06, -1.3556e-04],
        [-2.9337e-05, -6.1800e-05, -5.6997e-05,  ..., -9.5516e-05,
          1.3722e-05,  6.8221e-05],
        ...,
        [-2.8875e-05,  1.7340e-05,  7.1711e-05